<a href="https://colab.research.google.com/github/Geuens/curso_ai/blob/main/Copy_of_lanchain_chinook_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install langchain langchainhub langchain_openai openai
from langchain.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain import hub
from google.colab import userdata

secret_string = userdata.get('OPENAI_TOKEN')

# Inicializamos db
db = SQLDatabase.from_uri("sqlite:///chinook.db")
# Pull down prompt
prompt = hub.pull("rlm/text-to-sql")
# Initialize model
model = ChatOpenAI(openai_api_key=secret_string)

# Crear chain con LangChain Expression Language
inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}
sql_response = (
    inputs
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# Call with a given question
sql_response.invoke({"question": "Enumera las ventas totales por país. ¿Los clientes de qué país gastaron más?"})

'SQLQuery: \nSELECT c.country, SUM(i.total) AS total_sales\nFROM customer c\nJOIN invoice i ON c.customer_id = i.customer_id\nGROUP BY c.country\nORDER BY total_sales DESC\nLIMIT 1\n'